In [6]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import matplotlib.pyplot as plt  # data visualization
import seaborn as sns
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, max_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.cluster import KMeans
from sklearn.feature_selection import RFE
from sklearn.neighbors import KNeighborsRegressor  
from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split

report = {
        'model_type': [],
        'model_name': [],
        'rmse': [],
        'mae': [],
        'R2': [],
        'adjusted R2': []

    }
def evaluate(modeltype, modelname, Model, x_type, y_type, d, xlabel, ylabel, title, data__for_check):
   
    X, y = data__for_check[x_type].values.reshape(
        -1, 1), data__for_check[y_type].values.reshape(-1, 1)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2,  random_state=5)
    # making sure the same model is not re-entered again
    if modelname in report['model_name']:
        print("Prexisting Model")
        return 0
    poly_reg = PolynomialFeatures(degree=d)
   
    X_poly = poly_reg.fit_transform(X)
    # Fitting Model
    Model.fit(X_poly, data__for_check[y_type])

    # Predicting Values from test set using model
   
    x_grid = np.arange(min(X), max(X), 0.01)
    # Model Evaluation

    plt.figure(figsize=(7, 7))
    plt.scatter(data=data__for_check, x=x_type,
                y=y_type, color='darkblue')
    plt.plot(x_grid, Model.predict(
        poly_reg.fit_transform(x_grid.reshape(-1, 1))), color='red')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(modelname + title)
    plt.show()

# function to evaluate and update model and score


def evaluate_report(modeltype, modelname, Model, d, data__for_check, columns_data):
    data__for_check_filtered = data__for_check[columns_data]
    X, y = data__for_check_filtered.iloc[:,
                                         :-1], data__for_check_filtered.iloc[:, -1]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2,  random_state=5)
    # making sure the same model is not re-entered again
    if modelname in report['model_name']:
        print("Prexisting Model")
        return 0
    X_tr = X_train.copy()
    X_te = X_test.copy()
    poly_reg = PolynomialFeatures(degree=d)
    X_tr = poly_reg.fit_transform(X_tr)
    X_te = poly_reg.fit_transform(X_te)
    # Fitting Model
    Model.fit(X_tr, y_train)
    
    # Predicting Values from test set using model
    y_pred = Model.predict(X_te)
    # Model Evaluation

    # Mean Absolute Error
    mae = mean_absolute_error(y_test, y_pred)
    report['mae'].append(mae)  # Appending Metric

    # R2 score
    R2 = r2_score(y_test, y_pred)
    report['R2'].append(R2)  # Appending Metric

    # Root Mean Square Error
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    report['rmse'].append(rmse)  # Appending Metric

    # Adjusted R2 score
    adj_r2 = 1-(1-R2)*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1))
    report['adjusted R2'].append(adj_r2)  # Appending Metric

    # Appending Model Details
    report['model_name'].append(modelname)
    report['model_type'].append(modeltype)


# displays report in a dataframe
def display_report():
    return pd.DataFrame(report)